# Semantic Segmentation using TAO SegFormer Net

Transfer learning is the process of transferring learned features from one application to another. It is a commonly used training technique where you use a model trained on one task and re-train to use it on a different task. 

Train Adapt Optimize (TAO) Toolkit  is a simple and easy-to-use Python based AI toolkit for taking purpose-built AI models and customizing them with users' own data.

<img align="center" src="https://d29g4g2dyqv443.cloudfront.net/sites/default/files/akamai/TAO/tlt-tao-toolkit-bring-your-own-model-diagram.png" width="1080">


## Learning Objectives

In this notebook, you will learn how to leverage the simplicity and convenience of TAO to:

* Train a MIT-B1 SegFormer model on the ISBI dataset
* Evaluate the trained model.
* Run Inference on the trained model.
* Export the trained model to a .onnx file for deployment to DeepStream.

At the end of this notebook, you will have generated a trained and optimized `segformer` model
which you may deploy via [Triton](https://github.com/NVIDIA-AI-IOT/tao-toolkit-triton-apps)
or [DeepStream](https://developer.nvidia.com/deepstream-sdk).

## Table of Contents

This notebook shows an example usecase of SegFormer using Train Adapt Optimize (TAO) Toolkit.

0. [Set up env variables and map drives](#head-0)
1. [Installing the TAO launcher](#head-1)
2. [Preparing the dataset](#head-2)
3. [Provide training specification](#head-3)
4. [Run TAO training](#head-4)
5. [Evaluate trained models](#head-5)
6. [Inferences](#head-6)
7. [Deploy](#head-7)


### Sample prediction of SegFormer
<img align="center" src="https://github.com/vpraveen-nv/model_card_images/blob/main/cv/notebook/segformer/sample.jpg?raw=true" width="300" height="300">

## 0. Set up env variables and map drives <a class="anchor" id="head-0"></a>

The TAO launcher uses docker containers under the hood, and **for our data and results directory to be visible to the docker, they need to be mapped**. The launcher can be configured using the config file `~/.tao_mounts.json`. Apart from the mounts, you can also configure additional options like the Environment Variables and amount of Shared Memory available to the TAO launcher. <br>

`IMPORTANT NOTE:` The code below creates a sample `~/.tao_mounts.json`  file. Here, we can map directories in which we save the data, specs, results and cache. You should configure it for your specific case so these directories are correctly visible to the docker container.


In [ ]:
import os

# Please define this local project directory that needs to be mapped to the TAO docker session.
%env LOCAL_PROJECT_DIR=FIXME

os.environ["HOST_DATA_DIR"] = os.path.join(os.getenv("LOCAL_PROJECT_DIR", os.getcwd()), "data", "segformer")
os.environ["HOST_RESULTS_DIR"] = os.path.join(os.getenv("LOCAL_PROJECT_DIR", os.getcwd()), "segformer")

# Set this path if you don't run the notebook from the samples directory.
# %env NOTEBOOK_ROOT=/path/to/local/tao-experiments/segformer
# The sample spec files are present in the same path as the downloaded samples.
os.environ["HOST_SPECS_DIR"] = os.path.join(
    os.getenv("NOTEBOOK_ROOT", os.getcwd()),
    "specs"
)
# Point to the 'deps' folder in samples from where you are launching notebook inside segformer folder. `ngc-collaterals/cv/resource/notebooks/tao_launcher_starter_kit/`
%env PROJECT_DIR=FIXME
%env NUM_GPUS = 1

In [ ]:
! mkdir -p $HOST_DATA_DIR
! mkdir -p $HOST_SPECS_DIR
! mkdir -p $HOST_RESULTS_DIR

In [ ]:
# Mapping up the local directories to the TAO docker.
import json
import os
mounts_file = os.path.expanduser("~/.tao_mounts.json")
tlt_configs = {
   "Mounts":[
       # Mapping the data directory
       {
           "source": os.environ["LOCAL_PROJECT_DIR"],
           "destination": "/workspace/tao-experiments"
       },
       {
           "source": os.environ["HOST_DATA_DIR"],
           "destination": "/data"
       },
       {
           "source": os.environ["HOST_SPECS_DIR"],
           "destination": "/specs"
       },
       {
           "source": os.environ["HOST_RESULTS_DIR"],
           "destination": "/results"
       },
   ],
   "DockerOptions": {
        "shm_size": "16G",
        "ulimits": {
            "memlock": -1,
            "stack": 67108864
         }
   }
}
# Writing the mounts file.
with open(mounts_file, "w") as mfile:
    json.dump(tlt_configs, mfile, indent=4)

In [ ]:
!cat ~/.tao_mounts.json

## 1. Installing the TAO launcher <a class="anchor" id="head-1"></a>
The TAO launcher is a python package distributed as a python wheel listed in PyPI. You may install the launcher by executing the following cell.

Please note that TAO Toolkit recommends users to run the TAO launcher in a virtual env with python 3.6.9. You may follow the instruction in this [page](https://virtualenvwrapper.readthedocs.io/en/latest/install.html) to set up a python virtual env using the `virtualenv` and `virtualenvwrapper` packages. Once you have setup virtualenvwrapper, please set the version of python to be used in the virtual env by using the `VIRTUALENVWRAPPER_PYTHON` variable. You may do so by running

```sh
export VIRTUALENVWRAPPER_PYTHON=/path/to/bin/python3.x
```

where x >= 6 and <= 8

We recommend performing this step first and then launching the notebook from the virtual environment. In addition to installing TAO python package, please make sure of the following software requirements:
* python >=3.7, <=3.10.x
* docker-ce > 19.03.5
* docker-API 1.40
* nvidia-container-toolkit > 1.3.0-1
* nvidia-container-runtime > 3.4.0-1
* nvidia-docker2 > 2.5.0-1
* nvidia-driver > 455+

Once you have installed the pre-requisites, please log in to the docker registry nvcr.io by following the command below

```sh
docker login nvcr.io
```

You will be triggered to enter a username and password. The username is `$oauthtoken` and the password is the API key generated from `ngc.nvidia.com`. Please follow the instructions in the [NGC setup guide](https://docs.nvidia.com/ngc/ngc-overview/index.html#generating-api-key) to generate your own API key.


In [ ]:
# SKIP this step IF you have already installed the TAO launcher.
# !pip3 install nvidia-tao

In [ ]:
!tao info --verbose

## 2. Preparing the dataset <a class="anchor" id="head-2"></a>

### 2.1 Prepare dataset

We will be using the `ISBI Challenge: Segmentation of neuronal structures in EM stacks` dataset for the binary segmentation tutorial. Please access the open source repo here https://github.com/alexklibisz/isbi-2012/tree/master/data to download the data. The data is in .tif format. Copy the train-labels.tif, train-volume.tif, test-volume.tif files to `$HOST_DATA_DIR/`. 

In [ ]:
# Install the following dependencies for running the dataset preparation scripts
!pip3 install Cython==0.29.36
!pip3 install -r $PROJECT_DIR/deps/requirements-pip.txt

### A. Verify downloaded dataset <a class="anchor" id="head-1-1"></a>

In [ ]:
!ls -l $HOST_DATA_DIR

In [ ]:
# Check the .tiff files are present
!ls -l $HOST_DATA_DIR
!if [ ! -f $HOST_DATA_DIR/test-volume.tif ]; then echo 'test-volume.tif file not found, please download.'; else echo 'Found test-volume.tif file.';fi
!if [ ! -f $HOST_DATA_DIR/train-volume.tif ]; then echo 'train-volume.tif file not found, please download.'; else echo 'Found train-volume.tif file.';fi

### B.  Prepare ISBI data from TIF <a class="anchor" id="head-1-2"></a>

Prepare the images and masks from the .tif files by running the following script.

In [ ]:
!python prepare_data_isbi.py --input_dir=$HOST_DATA_DIR  --output_dir=$HOST_DATA_DIR

In [ ]:
!if [ -f $HOST_DATA_DIR/images/train/image_0.png ]; then echo 'Data prepared successfully !'; else echo 'Please verify Data Preparation.';fi
# Checking the files
!ls -l $HOST_DATA_DIR
!ls -l $HOST_DATA_DIR/images
!ls -l $HOST_DATA_DIR/masks
!ls -l $HOST_DATA_DIR/images/train
!ls -l $HOST_DATA_DIR/images/val
!ls -l $HOST_DATA_DIR/images/test

### C.  Visualize the Groundtruth Masks <a class="anchor" id="head-1-3"></a>

In [ ]:
# Run this cell to visualize the training data and the masks

!python vis_annotation_isbi.py -i $HOST_DATA_DIR/images/train -m $HOST_DATA_DIR/masks/train -o $HOST_RESULTS_DIR/isbi_experiment/vis_gt --num_classes 2 --num_images 10

In [ ]:
# Simple grid visualizer
%matplotlib inline

import matplotlib.pyplot as plt
import os
from math import ceil
valid_image_ext = ['.jpg', '.png']

def visualize_images(image_dir, result_dir, num_cols=4, num_images=10):
    output_path = os.path.join(os.environ['HOST_RESULTS_DIR'], result_dir, image_dir)
    if not os.path.isdir(output_path):
        os.makedirs(output_path)
    num_rows = int(ceil(float(num_images) / float(num_cols)))
    f, axarr = plt.subplots(num_rows, num_cols, figsize=[80,30])
    f.tight_layout()
    a = [os.path.join(output_path, image) for image in os.listdir(output_path) 
         if os.path.splitext(image)[1].lower() in valid_image_ext]
    for idx, img_path in enumerate(a[:num_images]):
        col_id = idx % num_cols
        row_id = idx // num_cols
        img = plt.imread(img_path)
        axarr[row_id, col_id].imshow(img) 

In [ ]:
# Visualizing the sample images.
OUTPUT_PATH = 'vis_gt' # relative path from $USER_EXPERIMENT_DIR.
COLS = 2 # number of columns in the visualizer grid.
IMAGES = 4 # number of images to visualize.

visualize_images(OUTPUT_PATH, "isbi_experiment", num_cols=COLS, num_images=IMAGES)

## 3. Provide training specification <a class="anchor" id="head-2"></a>

We provide specification files to configure the training parameters including:

* checkpoint_config: configure the checkpoint setting
    * interval: number of iterations at which checkpoint needs to be saved
* train_config: configure the training hyperparameters
    * optim_config
    * epochs
    * checkpoint_interval
* dataset_config: configure the dataset and augmentation methods
    * train_img_dirs
    * train_ann_dirs
    * pallete: color and mapping class for each class
    * output_shape
    * batch_size
    * workers: number of workers to do data loading
    * clips_per_video: number of clips to be sampled from single video
    * augmentation_config

Please refer to the TAO documentation about Segformer to get all the parameters that are configurable.

In [ ]:
!cat $HOST_SPECS_DIR/train_isbi.yaml

## 4. Run TAO training <a class="anchor" id="head-3"></a>
* Provide the sample spec file and the output directory location for models
* WARNING: training will take several hours or one day to complete

In [ ]:
# NOTE: The following paths are set from the perspective of the TAO Docker.

# The data is saved here
%env DATA_DIR = /data
%env SPECS_DIR = /specs
%env RESULTS_DIR = /results

In [ ]:
print("Train SegFormer Model")
!tao model segformer train \
                  -e $SPECS_DIR/train_isbi.yaml \
                  results_dir=$RESULTS_DIR/isbi_experiment \
                  train.num_gpus=$NUM_GPUS 

In [ ]:
print('Checkpoints:')
print('---------------------')
!ls -ltrh $HOST_RESULTS_DIR/isbi_experiment/train

In [ ]:
print('Rename a model: Note that the training is not deterministic, so you may change the model name accordingly.')
print('---------------------')
# NOTE: The following command may require `sudo`. You can run the command outside the notebook.
!find $HOST_RESULTS_DIR/isbi_experiment/train -name "iter_1000.pth" | xargs realpath | xargs -I {} mv {} $HOST_RESULTS_DIR/isbi_experiment/train/isbi_model.pth 
!ls -ltrh $HOST_RESULTS_DIR/isbi_experiment/train/isbi_model.pth

## 5. Evaluate trained models <a class="anchor" id="head-4"></a>


Evaluate ISBI Model

In [ ]:
!tao model segformer evaluate \
                    -e $SPECS_DIR/test_isbi.yaml \
                    evaluate.checkpoint=$RESULTS_DIR/isbi_experiment/train/isbi_model.pth \
                    results_dir=$RESULTS_DIR/isbi_experiment

## 6. Inferences <a class="anchor" id="head-5"></a>
In this section, we run the segformer inference tool to generate inferences with the trained RGB models and print the results. 


### 6.1. Run Inference

In [ ]:
!tao model segformer inference \
                    -e $SPECS_DIR/test_isbi.yaml \
                    inference.checkpoint=$RESULTS_DIR/isbi_experiment/train/isbi_model.pth \
                    results_dir=$RESULTS_DIR/isbi_experiment

### 6.2. Visualize the Inference

In [ ]:
!ls -ltrh $HOST_RESULTS_DIR/isbi_experiment/inference

In [ ]:
# Visualizing the sample images.
OUTPUT_PATH = 'vis_tao' # relative path from $USER_EXPERIMENT_DIR.
COLS = 2 # number of columns in the visualizer grid.
IMAGES = 4 # number of images to visualize.

visualize_images(OUTPUT_PATH, "isbi_experiment/inference", num_cols=COLS, num_images=IMAGES)

## 7. Deploy! <a class="anchor" id="head-6"></a>

In [ ]:
!mkdir -p $HOST_RESULTS_DIR/isbi_experiment/export

In [ ]:
# Export the ISBI model to encrypted ONNX model
# NOTE: Export is done on single GPU - GPU num need not be provided

!tao model segformer export \
                   -e $SPECS_DIR/export_isbi.yaml \
                   export.checkpoint=$RESULTS_DIR/isbi_experiment/train/isbi_model.pth\
                   results_dir=$RESULTS_DIR/isbi_experiment/ \
                   export.onnx_file=$RESULTS_DIR/isbi_experiment/export/isbi_model_export.onnx

In [ ]:
print('Exported model:')
print('------------')
!ls -lth $HOST_RESULTS_DIR/isbi_experiment/export

In [ ]:
!tao deploy -h

In [ ]:
# Generate TensorRT engine using tao deploy
!tao deploy segformer gen_trt_engine -e $SPECS_DIR/export_isbi.yaml \
                                     results_dir=$RESULTS_DIR/isbi_experiment/export/ \
                                     gen_trt_engine.onnx_file=$RESULTS_DIR/isbi_experiment/export/isbi_model_export.onnx \
                                     gen_trt_engine.trt_engine=$RESULTS_DIR/isbi_experiment/export/isbi_model_export.engine

In [ ]:
!mkdir -p $HOST_RESULTS_DIR/isbi_experiment/trt_evaluate
# Evaluate with generated TensorRT engine
!tao deploy segformer evaluate -e $SPECS_DIR/test_isbi.yaml \
                               evaluate.trt_engine=$RESULTS_DIR/isbi_experiment/export/isbi_model_export.engine \
                               results_dir=$RESULTS_DIR/isbi_experiment/trt_evaluate/ 

In [ ]:
!mkdir -p $HOST_RESULTS_DIR/isbi_experiment/trt_infer
# Inference with generated TensorRT engine
!tao deploy segformer inference -e $SPECS_DIR/test_isbi.yaml \
                                inference.trt_engine=$RESULTS_DIR/isbi_experiment/export/isbi_model_export.engine \
                                results_dir=$RESULTS_DIR/isbi_experiment/trt_infer/ 

This notebook has come to an end.